In [ ]:
import cv2
import imutils
import numpy as np
import math
import os

import matplotlib.pyplot as plt
%matplotlib qt

white = [255,255,255]

def blank(imagem):
    r, c = imagem.shape
    for i in range(r):
        for j in range(c):
            if (imagem[i][j] != 0):
                return False
            
    return True

def lastPoint(imagem):
    out = imagem.copy()
    while(True):
        res = cv2.erode(out, cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3)))
        if blank(res):
            #print("Teste")
            r, c = imagem.shape
            for i in range(r):
                for j in range(c):
                    if (out[i,j]==white).all():
                        return i, j, out
        else:
            out = res
            
    #return out
    
def createSeries(path):
    mat = cv2.imread(path)  
    print(path)

    mRows, mCols, mType = mat.shape
    if mRows>mCols:
        mat = imutils.resize(mat, width=300)
    else:
        mat = imutils.resize(mat, height=300)

    img = mat.copy()

    b,g,r = cv2.split(mat)
    linhas, colunas = g.shape
    newTest = g.copy()
    pixel = 0
    for i in range(linhas):
        for j in range(colunas):
        
            pixel = int(g[i,j]) - int(r[i,j])
            if pixel<20:
                newTest[i,j] = 0
            else:
                newTest[i,j] = 2*pixel

    contour = g.copy()
    mask = g.copy()
    contours = cv2.findContours(newTest, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
 
    for i in range(linhas):
        for j in range(colunas):
            contour[i,j] = 0
            mask[i,j] = 0

    cnts = imutils.grab_contours(contours)
    c = max(cnts, key=cv2.contourArea)        
    cv2.drawContours(contour, [c], -1, (255, 0, 0),  2)
    cv2.drawContours(mask, [c], -1, (255, 0, 0),  -1)
    
    #img = cv2.bitwise_and(r, mask)

    #img = cv2.Canny(img,250,350)
#     cv2.imshow("point", img)
#     cv2.waitKey(1)
 
    #img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT,(9,9)))

    #y, x, img = lastPoint(img)
    x,y,w,h = cv2.boundingRect(c)
    
    cv2.rectangle(contour,(x,y),(x+w,y+h),(255,255,255),2)
    
    
    x = int(x+w/2)
    y = int(y+h/2)

    graph = []
    cont = 0
    red = [255, 0, 0]
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    p, = ax.plot(np.linspace(0,1))

    # First display
    plt.show()
    
    plt.draw()
    
    xAxis = []
    for v in c:
        for p in v:
            lines = contour.copy()
            d = int(math.sqrt(math.pow(p[0]-x,2)+math.pow(p[1]-y,2)))
            
            cv2.line(lines, (x, y), (p[0], p[1]), red, 2)
            cv2.imshow("Lines", lines)
            cv2.waitKey(1)
            
            graph.append(d)
            
            xAxis.append(cont)
            
            ax.cla()
            ax.plot(xAxis, graph, color='tab:blue')
            plt.draw()
            #plt.pause(0.05)
            cont +=1
            
    return graph

def myListDir(directory):
    filelist = os.listdir(directory)
    filelist = sorted(filelist)
    return [x for x in filelist
            if not (x.startswith('.'))]

def rollVector(vec):
    maior = vec[0]
    value = 0
    i = 0
    for v in vec:
        if v > maior:
            maior = v
            value = i
        i += 1
    return np.roll(vec, -value)

path = "/Users/joseluis/Desktop/Database/train" 
with open("/Users/joseluis/Desktop/Database/rollVector.txt", 'w') as f:
    for folders in myListDir(path):
        
        folder = os.path.join(path, folders)
        
        for filename in myListDir(folder):
            file = os.path.join(folder, filename)
            series = createSeries(file)
            rolled = rollVector(series)
            norm = rolled / np.amax(rolled)
            seriesStr = " ".join(str("{0:.2f}".format(i)) for i in norm)
            f.write(seriesStr+"\n")
            
print("Ok!")
cv2.destroyAllWindows()

In [ ]:
# import cv2
# import imutils
# import numpy as np
# import math
# import os

# import matplotlib.pyplot as plt
# %matplotlib qt

# white = [255,255,255]

# def blank(imagem):
#     r, c = imagem.shape
#     for i in range(r):
#         for j in range(c):
#             if (imagem[i][j] != 0):
#                 return False
            
#     return True

# def lastPoint(imagem):
#     out = imagem.copy()
#     while(True):
#         res = cv2.erode(out, cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3)))
#         if blank(res):
#             r, c = imagem.shape
#             for i in range(r):
#                 for j in range(c):
#                     if (out[i,j]==white).all():
#                         return i, j, out
#         else:
#             out = res
            
#     return out

# def createSeries(path):
#     mat = cv2.imread(path)
    
#     print(path)
    
#     mRows, mCols, mType = mat.shape
#     if mRows>mCols:
#         mat = imutils.resize(mat, width=300)
#     else:
#         mat = imutils.resize(mat, height=300)
    
#     img = mat.copy()
#     cv2.imshow("Original", img)
#     cv2.waitKey(1)

#     sharpen = mat.copy()
#     gaussian = cv2.GaussianBlur(sharpen, (9,9), 10.0)
#     sharpen = cv2.addWeighted(sharpen, 1.5, gaussian, -0.5, 0, sharpen)

#     b,g,r = cv2.split(mat)

#     linhas, colunas = g.shape
#     newTest = g.copy()
#     pixel = 0
#     for i in range(linhas):
#         for j in range(colunas):
        
#             pixel = int(g[i,j]) - int(r[i,j])
#             if pixel<20:
#                 newTest[i,j] = 0
#             else:
#                 newTest[i,j] = 2*pixel
                
#     contour = g.copy()
#     contours = cv2.findContours(newTest, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
#     for i in range(linhas):
#         for j in range(colunas):
#             contour[i,j] = 0

#     cnts = imutils.grab_contours(contours)

#     c = max(cnts, key=cv2.contourArea)        
#     cv2.drawContours(contour, [c], -1, (255, 0, 0), 2)

#     ret,img = cv2.threshold(r,220,255,cv2.THRESH_BINARY)
    
#     y, x, img = lastPoint(img)
    
# #     cv2.imshow("Point", img)
# #     cv2.waitKey(1)
    
#     graph = []
#     cont = 0
#     red = [255, 0, 0]
    
#     fig = plt.figure()
#     ax = fig.add_subplot(111)
#     p, = ax.plot(np.linspace(0,1))

#     # First display
#     plt.show()
    
#     plt.draw()
    
#     xAxis = []
#     for v in c:
#         for p in v:
#             lines = contour.copy()
#             d = int(math.sqrt(math.pow(p[0]-x,2)+math.pow(p[1]-y,2)))
            
#             cv2.line(lines, (x, y), (p[0], p[1]), red, 2)
#             cv2.imshow("Lines", lines)
#             cv2.waitKey(1)
            
#             graph.append(d)
            
#             xAxis.append(cont)
            
#             ax.cla()
#             ax.plot(xAxis, graph, color='tab:blue')
#             plt.draw()
#             #plt.pause(0.05)
#             cont +=1
            
#     return graph 

# def myListDir(directory):
#     filelist = os.listdir(directory)
#     filelist = sorted(filelist)
#     return [x for x in filelist
#             if not (x.startswith('.'))]

# def rollVector(vec):
#     maior = vec[0]
#     value = 0
#     i = 0
#     for v in vec:
#         if v > maior:
#             maior = v
#             value = i
#         i += 1
#     return np.roll(vec, -value)


# path = "/Users/joseluis/Desktop/Database/train" 
# with open("/Users/joseluis/Desktop/Database/rollVector.txt", 'w') as f:
#     for folders in myListDir(path):
        
#         folder = os.path.join(path, folders)
        
#         for filename in myListDir(folder):
#             file = os.path.join(folder, filename)
#             series = createSeries(file)
#             rolled = rollVector(series)
#             norm = rolled / np.amax(rolled)
#             seriesStr = " ".join(str("{0:.2f}".format(i)) for i in norm)
#             f.write(seriesStr+"\n")
            
# print("Ok!")
# cv2.destroyAllWindows()

In [ ]:
import cv2
import imutils
import numpy as np
import math
import os

import matplotlib.pyplot as plt
%matplotlib qt

white = [255,255,255]

def blank(imagem):
    r, c = imagem.shape
    for i in range(r):
        for j in range(c):
            if (imagem[i][j] != 0):
                return False
            
    return True

def lastPoint(imagem):
    out = imagem.copy()
    while(True):
        res = cv2.erode(out, cv2.getStructuringElement(cv2.MORPH_CROSS,(3,3)))
        if blank(res):
            #print("Teste")
            r, c = imagem.shape
            for i in range(r):
                for j in range(c):
                    if (out[i,j]==white).all():
                        return i, j, out
        else:
            out = res
            
    #return out
    
def createSeries(path):
    mat = cv2.imread(path)  
    print(path)

    mRows, mCols, mType = mat.shape
    if mRows>mCols:
        mat = imutils.resize(mat, width=300)
    else:
        mat = imutils.resize(mat, height=300)

    img = mat.copy()

    b,g,r = cv2.split(mat)
    linhas, colunas = g.shape
    newTest = g.copy()
    pixel = 0
    for i in range(linhas):
        for j in range(colunas):
        
            pixel = int(g[i,j]) - int(r[i,j])
            if pixel<20:
                newTest[i,j] = 0
            else:
                newTest[i,j] = 2*pixel

    contour = g.copy()
    mask = g.copy()
    contours = cv2.findContours(newTest, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
 
    for i in range(linhas):
        for j in range(colunas):
            contour[i,j] = 0
            mask[i,j] = 0

    cnts = imutils.grab_contours(contours)
    c = max(cnts, key=cv2.contourArea)        
    cv2.drawContours(contour, [c], -1, (255, 0, 0),  2)
    cv2.drawContours(mask, [c], -1, (255, 0, 0),  -1)
    
    #img = cv2.bitwise_and(r, mask)

    #img = cv2.Canny(img,250,350)
#     cv2.imshow("point", img)
#     cv2.waitKey(1)
 
    #img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT,(9,9)))

    #y, x, img = lastPoint(img)
    
    x,y,w,h = cv2.boundingRect(c)
    x = int((x+w)/2)
    y = int((y+h)/2)

    graph = []
    cont = 0
    red = [255, 0, 0]
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    p, = ax.plot(np.linspace(0,1))

    # First display
    plt.show()
    
    plt.draw()
    
    xAxis = []
    for v in c:
        for p in v:
            lines = contour.copy()
            d = int(math.sqrt(math.pow(p[0]-x,2)+math.pow(p[1]-y,2)))
            
            cv2.line(lines, (x, y), (p[0], p[1]), red, 2)
            cv2.imshow("Lines", lines)
            cv2.waitKey(1)
            
            graph.append(d)
            
            xAxis.append(cont)
            
            ax.cla()
            ax.plot(xAxis, graph, color='tab:blue')
            plt.draw()
            #plt.pause(0.05)
            cont +=1
            
    return graph

def myListDir(directory):
    filelist = os.listdir(directory)
    filelist = sorted(filelist)
    return [x for x in filelist
            if not (x.startswith('.'))]

def rollVector(vec):
    maior = vec[0]
    value = 0
    i = 0
    for v in vec:
        if v > maior:
            maior = v
            value = i
        i += 1
    return np.roll(vec, -value)

path = "/Users/joseluis/Desktop/Database/valid" 
with open("/Users/joseluis/Desktop/Database/rollValid.txt", 'w') as f:
    for filename in myListDir(path):
        file = os.path.join(path, filename)
        series = createSeries(file)
        rolled = rollVector(series)
        norm = rolled / np.amax(rolled)
        seriesStr = " ".join(str("{0:.2f}".format(i)) for i in norm)
        f.write(seriesStr+"\n")
        
print("Ok!")
cv2.destroyAllWindows()